# 通信会社 ネットワーク品質 AI エージェント ハンズオン

このハンズオンでは、Snowflake の AI 機能を使って**エリアマスタ、通信品質データ、設備キャパシティデータ、気象データ**の整備から **「自然言語で質問できる AI エージェント作成」** までを一気通貫で構築します。

### 今日つくるもの

携帯電話通信会社のネットワーク品質データ（通信速度・遅延・接続成功率・トラフィック量など）を横断的に分析できる AI エージェントです。  
完成すると、例えばこんな質問に自然言語で答えてくれます：

> 「2025年の東京エリアの通信速度推移を月別で可視化して。気温との相関も見たい」  
> 「現在設備が逼迫しているエリアと、そのトラフィック推移を教えて」  
> 「関東地方のエリア一覧とそれぞれの特徴を教えて」  
> 「雨の日と晴れの日で通信品質にどんな違いがある？」

---

### ハンズオンの流れ（所要時間：約80分 ＋ 応用25分）

| Step | やること | 学べる Snowflake 機能 | 所要時間 |
|:----:|----------|----------------------|:--------:|
| 1 | AI でエリアマスタを整備 | AI_CLASSIFY / AI_COMPLETE | 10分 |
| 2 | 気象データを取り込む | Snowflake Marketplace | 10分 |
| 3 | 通信品質の予兆検知（ML） | ML Functions（ANOMALY_DETECTION / FORECAST / TOP_INSIGHTS） | 20分 |
| 4 | エリアの意味検索を可能にする | Cortex Search Service | 10分 |
| 5 | データの意味を定義する | Semantic View（Cortex Analyst） | 20分 |
| 6 | AI エージェントを組み立てる | Cortex Agent | 15分 |
| 7 | 動かしてみよう！ | Cortex Agent UI | 5分 |
| 8 | 【応用】カスタムモデルで予兆検知 | Snowpark ML / Container Runtime / Model Registry | 25分 |

> **Note**: Step 8 は応用編です。時間に余裕がある場合にチャレンジしてください。Step 1〜7 だけでも十分にハンズオンは完結します。

## Step 0: 事前準備
使用するデータベース・スキーマ・ウェアハウスを設定します。

In [ ]:
USE DATABASE TELECOM_AI_HANDSON;
USE SCHEMA ANALYTICS;
USE WAREHOUSE COMPUTE_WH;

---
## Step 1: AI でエリアマスタを整備する

現在のエリアマスタ（`RAW.AREA_MASTER`）には、エリアID・エリア名・地方名しかありません。  
ここに Snowflake の **AI SQL 関数** を使って、2つの情報を自動で追加します。

### 追加する情報

#### 1. エリアタイプの自動分類 — `AI_CLASSIFY`

エリア名だけを見て、AI が自動的にエリアタイプを判定します。  
あらかじめ定義した **5つのタイプ** に分類します（MECE＝漏れなくダブりなく）。

| エリアタイプ | どんなエリア？ |
|-------------|---------------|
| 都心ビジネス | オフィス街、駅前商業地区、高層ビル密集地 |
| 商業・繁華街 | ショッピングモール、繁華街、娯楽施設集中地 |
| 住宅地 | マンション・戸建て住宅が中心の生活エリア |
| 観光・ウォーターフロント | 観光地、海辺、リゾート、歴史的エリア |
| 郊外・地方 | 工業地帯、農村部、山間地、人口密度が低いエリア |

#### 2. エリア特性説明文の自動生成 — `AI_COMPLETE`

エリア名と地方名から、そのエリアの **通信需要の特性や課題を説明する文章（約300文字）** を AI が自動で生成します。

> **ポイント**: `AI_CLASSIFY` はラベル＋説明文を渡すだけで分類してくれるので、従来のルールベースのロジックを書く必要がありません。

In [ ]:
SELECT * FROM TELECOM_AI_HANDSON.RAW.AREA_MASTER;

それでは実行しましょう。エリア数にもよりますが、**約1〜2分** で完了します。

In [ ]:
CREATE OR REPLACE TABLE TELECOM_AI_HANDSON.ANALYTICS.AI_AREA_MASTER AS
SELECT 
    AREA_ID,
    AREA_NAME,
    REGION_NAME,
    REGION_CODE,
    -- ① AI_CLASSIFY関数でエリアタイプを分類（MECE）
    AI_CLASSIFY(
        AREA_NAME,
        [
            {'label': '都心ビジネス', 'description': 'オフィス街、駅前商業地区、高層ビル密集地、ビジネス中心地'},
            {'label': '商業・繁華街', 'description': 'ショッピングモール、繁華街、娯楽施設集中地、歓楽街'},
            {'label': '住宅地', 'description': 'マンション・戸建て住宅が中心の生活エリア、ベッドタウン'},
            {'label': '観光・ウォーターフロント', 'description': '観光地、海辺、リゾート、歴史的エリア、臨海地区、湾岸'},
            {'label': '郊外・地方', 'description': '工業地帯、農村部、山間地、人口密度が低いエリア、地方都市郊外'}
        ],
        {'task_description': '日本の携帯電話通信エリア名から最も適切なエリアタイプに分類してください'}
    ):labels[0]::VARCHAR AS AREA_TYPE,
    -- ② AI_COMPLETE関数でエリア特性説明文生成
    TRIM(SNOWFLAKE.CORTEX.AI_COMPLETE(
        'llama3.1-405b',
        '以下の携帯電話通信エリアについて、通信需要の特性や課題を説明する文章（300文字程度）を日本語で1つ作成してください。
         エリア名: ' || AREA_NAME || '
         地方: ' || COALESCE(REGION_NAME, '不明') || '
         説明文:'
    )) AS AREA_DESCRIPTION,
    CURRENT_TIMESTAMP() AS CREATED_AT
FROM TELECOM_AI_HANDSON.RAW.AREA_MASTER;

### 結果を見てみましょう

AI が生成した `AREA_TYPE`（エリアタイプ）と `AREA_DESCRIPTION`（エリア特性説明文）を確認します。

In [ ]:
SELECT * FROM TELECOM_AI_HANDSON.ANALYTICS.AI_AREA_MASTER;

### エリアタイプ分類の精度を確認

5つのエリアタイプにバランスよく分類されているか確認しましょう。  
偏りがある場合は、`AI_CLASSIFY` に渡すエリアタイプの `description` を調整することで精度を改善できます。

In [ ]:
SELECT 
    AREA_TYPE,
    COUNT(*) AS AREA_COUNT
FROM TELECOM_AI_HANDSON.ANALYTICS.AI_AREA_MASTER
GROUP BY AREA_TYPE
ORDER BY AREA_COUNT DESC;

---
## Step 2: 気象データを取り込む（Snowflake Marketplace）

通信品質は天候に大きく左右されます。  
雨天時は電波の減衰が増し通信品質が低下、台風時はさらに顕著 — こうした **天候×通信品質の分析** を可能にするため、  
Snowflake Marketplace の **気象データ** を取り込みます。

### データソース

**Weather Source LLC: Frostbyte** が提供するグローバル気象データを使用します。  
日本（JP）の日次気象データを 2019年〜 取得できます。

### なぜテーブルを作るのか？

元データは華氏・インチ単位で、グローバルデータが含まれています。  
通信品質データと結合するには **日本のデータのみを抽出し、メートル法に変換して日付ごとに1行** にする必要があるため、  
全国平均に集約するテーブルを作成します。

| 集約カラム | 計算方法 | 説明 |
|------------|----------|------|
| 平均気温 | AVG | 全国の観測所の平均（℃に変換） |
| 最高気温 | MAX | 全国で最も高かった気温（℃に変換） |
| 最低気温 | MIN | 全国で最も低かった気温（℃に変換） |
| 降水量 | AVG | 全国平均の降水量（mmに変換） |
| 湿度 | AVG | 全国平均の湿度 |
| 日照時間 | AVG | 全国平均の日射量 |
| 降雪量 | SUM | 全国の降雪量合計（cmに変換） |
| 風速 | AVG | 全国平均の風速（km/hに変換） |

In [ ]:
CREATE OR REPLACE TABLE TELECOM_AI_HANDSON.ANALYTICS.DAILY_WEATHER AS
SELECT
    DATE_VALID_STD                                                      AS WEATHER_DATE,
    ROUND(AVG((AVG_TEMPERATURE_AIR_2M_F - 32) * 5/9), 1)                AS AVG_TEMPERATURE,
    ROUND(MAX((MAX_TEMPERATURE_AIR_2M_F - 32) * 5/9), 1)                AS MAX_TEMPERATURE,
    ROUND(MIN((MIN_TEMPERATURE_AIR_2M_F - 32) * 5/9), 1)                AS MIN_TEMPERATURE,
    ROUND(AVG(TOT_PRECIPITATION_IN * 25.4), 1)                          AS AVG_RAINFALL,
    ROUND(AVG(AVG_HUMIDITY_RELATIVE_2M_PCT), 1)                         AS AVG_HUMIDITY,
    ROUND(AVG(AVG_RADIATION_SOLAR_TOTAL_WPM2), 2)                       AS AVG_SUNLIGHT_HOURS,
    ROUND(SUM(TOT_SNOWFALL_IN * 25.4), 1)                               AS TOTAL_SNOWFALL,
    ROUND(AVG(AVG_WIND_SPEED_10M_MPH * 1.60934), 1)                     AS AVG_WIND_SPEED
FROM WEATHER_SOURCE_LLC_FROSTBYTE.ONPOINT_ID.HISTORY_DAY
WHERE COUNTRY = 'JP'
GROUP BY DATE_VALID_STD
ORDER BY DATE_VALID_STD;

In [ ]:
SELECT * FROM TELECOM_AI_HANDSON.ANALYTICS.DAILY_WEATHER;

---
## Step 3: 通信品質の予兆検知（ML Functions）

Step 2 で取り込んだ気象データと通信品質データを組み合わせて、**SQL だけで機械学習の予兆検知パイプライン** を構築します。

### この Step で学ぶこと

| パート | やること | 使う ML Function |
|:------:|----------|:----------------:|
| A | 切断率の異常を検知する | `ANOMALY_DETECTION` |
| B | トラフィック量の将来予測 | `FORECAST` |
| C | 品質劣化の要因を特定する | `TOP_INSIGHTS` |

### ML Functions とは？

Snowflake の **ML Functions** は、SQL だけで機械学習モデルの訓練・推論ができる機能です。  
Python の知識は不要で、`CREATE SNOWFLAKE.ML.ANOMALY_DETECTION` のように SQL 文でモデルを作成できます。  
内部的には Gradient Boosting Machine（GBM）や ARIMA 等のアルゴリズムが使われています。

> **ポイント**: ML Functions は Standard Edition 以上の全エディションで利用可能です。

### パート A: 切断率の異常検知（ANOMALY_DETECTION）

通信品質データの中でも **切断率（CALL_DROP_RATE）** は、値が跳ね上がるとユーザー体験に直結する重要指標です。  
ここでは、全国平均の日次切断率を使って異常検知モデルを訓練し、品質劣化イベントを自動検出します。

#### ステップ 1: 訓練データとテストデータを準備

- **訓練データ**: 2022年〜2024年（3年分の通常パターンを学習）
- **テストデータ**: 2025年〜（異常を検知する対象期間）

In [ ]:
-- 訓練データ: 全国平均の日次切断率（2022〜2024年）
CREATE OR REPLACE VIEW TELECOM_AI_HANDSON.ANALYTICS.NQ_TRAIN_DATA AS
SELECT
    MEASURE_DATE::TIMESTAMP_NTZ AS TS,
    AVG(CALL_DROP_RATE)::FLOAT AS CALL_DROP_RATE
FROM TELECOM_AI_HANDSON.ANALYTICS.NETWORK_QUALITY
WHERE MEASURE_DATE BETWEEN '2022-01-01' AND '2024-12-31'
GROUP BY MEASURE_DATE
ORDER BY MEASURE_DATE;

-- テストデータ: 2025年以降
CREATE OR REPLACE VIEW TELECOM_AI_HANDSON.ANALYTICS.NQ_TEST_DATA AS
SELECT
    MEASURE_DATE::TIMESTAMP_NTZ AS TS,
    AVG(CALL_DROP_RATE)::FLOAT AS CALL_DROP_RATE
FROM TELECOM_AI_HANDSON.ANALYTICS.NETWORK_QUALITY
WHERE MEASURE_DATE >= '2025-01-01'
GROUP BY MEASURE_DATE
ORDER BY MEASURE_DATE;

-- 確認
SELECT 'TRAIN' AS DATASET, COUNT(*) AS DAYS, MIN(TS) AS START_DATE, MAX(TS) AS END_DATE FROM TELECOM_AI_HANDSON.ANALYTICS.NQ_TRAIN_DATA
UNION ALL
SELECT 'TEST', COUNT(*), MIN(TS), MAX(TS) FROM TELECOM_AI_HANDSON.ANALYTICS.NQ_TEST_DATA;

#### ステップ 2: 基本の異常検知モデルを訓練

まずは外因変数なしの **教師なし（Unsupervised）** モデルを作成します。  
切断率の時系列パターンだけを学習し、そこから外れるデータを「異常」と判定します。

In [ ]:
-- 基本モデルの訓練（教師なし、外因変数なし）
CREATE OR REPLACE SNOWFLAKE.ML.ANOMALY_DETECTION TELECOM_AI_HANDSON.ANALYTICS.NQ_ANOMALY_BASE(
    INPUT_DATA => TABLE(TELECOM_AI_HANDSON.ANALYTICS.NQ_TRAIN_DATA),
    TIMESTAMP_COLNAME => 'TS',
    TARGET_COLNAME => 'CALL_DROP_RATE',
    LABEL_COLNAME => ''
);

In [ ]:
-- テストデータに対して異常を検知
CREATE OR REPLACE TABLE TELECOM_AI_HANDSON.ANALYTICS.ANOMALY_RESULTS AS
SELECT * FROM TABLE(TELECOM_AI_HANDSON.ANALYTICS.NQ_ANOMALY_BASE!DETECT_ANOMALIES(
    INPUT_DATA => TABLE(TELECOM_AI_HANDSON.ANALYTICS.NQ_TEST_DATA),
    TIMESTAMP_COLNAME => 'TS',
    TARGET_COLNAME => 'CALL_DROP_RATE',
    CONFIG_OBJECT => {'prediction_interval': 0.95}
));

-- 検知された異常日を確認
SELECT
    TS::DATE AS ANOMALY_DATE,
    ROUND(Y, 4) AS ACTUAL_DROP_RATE,
    ROUND(FORECAST, 4) AS EXPECTED_DROP_RATE,
    ROUND(UPPER_BOUND, 4) AS UPPER_BOUND,
    ROUND(Y - UPPER_BOUND, 4) AS EXCESS
FROM TELECOM_AI_HANDSON.ANALYTICS.ANOMALY_RESULTS
WHERE IS_ANOMALY = TRUE
ORDER BY TS;

#### ステップ 3: カスタム — 気象データを外因変数として追加

基本モデルは切断率の時系列パターンのみで判断しますが、**降水量や風速が高い日は電波の減衰が増し、切断率が上がりやすい** ことがわかっています。  
気象データを **外因変数（Exogenous Variables）** としてモデルに追加し、天候を考慮した異常検知に改善しましょう。

> **外因変数とは？**: モデルが予測に使う「追加ヒント」です。タイムスタンプとターゲット以外のカラムは、すべて自動的に外因変数として扱われます。

In [ ]:
-- 気象データ付き訓練データ（降水量・風速を外因変数として追加）
CREATE OR REPLACE VIEW TELECOM_AI_HANDSON.ANALYTICS.NQ_TRAIN_WITH_WEATHER AS
SELECT
    nq.MEASURE_DATE::TIMESTAMP_NTZ AS TS,
    AVG(nq.CALL_DROP_RATE)::FLOAT AS CALL_DROP_RATE,
    MAX(w.AVG_RAINFALL)::FLOAT AS AVG_RAINFALL,
    MAX(w.AVG_WIND_SPEED)::FLOAT AS AVG_WIND_SPEED
FROM TELECOM_AI_HANDSON.ANALYTICS.NETWORK_QUALITY nq
JOIN TELECOM_AI_HANDSON.ANALYTICS.DAILY_WEATHER w
    ON nq.MEASURE_DATE = w.WEATHER_DATE
WHERE nq.MEASURE_DATE BETWEEN '2022-01-01' AND '2024-12-31'
GROUP BY nq.MEASURE_DATE
ORDER BY nq.MEASURE_DATE;

-- 気象データ付きテストデータ
CREATE OR REPLACE VIEW TELECOM_AI_HANDSON.ANALYTICS.NQ_TEST_WITH_WEATHER AS
SELECT
    nq.MEASURE_DATE::TIMESTAMP_NTZ AS TS,
    AVG(nq.CALL_DROP_RATE)::FLOAT AS CALL_DROP_RATE,
    MAX(w.AVG_RAINFALL)::FLOAT AS AVG_RAINFALL,
    MAX(w.AVG_WIND_SPEED)::FLOAT AS AVG_WIND_SPEED
FROM TELECOM_AI_HANDSON.ANALYTICS.NETWORK_QUALITY nq
JOIN TELECOM_AI_HANDSON.ANALYTICS.DAILY_WEATHER w
    ON nq.MEASURE_DATE = w.WEATHER_DATE
WHERE nq.MEASURE_DATE >= '2025-01-01'
GROUP BY nq.MEASURE_DATE
ORDER BY nq.MEASURE_DATE;

-- 気象データ付きモデルの訓練
CREATE OR REPLACE SNOWFLAKE.ML.ANOMALY_DETECTION TELECOM_AI_HANDSON.ANALYTICS.NQ_ANOMALY_WEATHER(
    INPUT_DATA => TABLE(TELECOM_AI_HANDSON.ANALYTICS.NQ_TRAIN_WITH_WEATHER),
    TIMESTAMP_COLNAME => 'TS',
    TARGET_COLNAME => 'CALL_DROP_RATE',
    LABEL_COLNAME => ''
);

#### ステップ 4: 特徴量の重要度を確認

`!EXPLAIN_FEATURE_IMPORTANCE()` で、モデルが異常検知にどの特徴量をどれだけ使っているかを確認します。  
降水量や風速が上位に来ていれば、天候が切断率に影響していることをモデルが学習できています。

In [ ]:
-- 特徴量の重要度を確認
CALL TELECOM_AI_HANDSON.ANALYTICS.NQ_ANOMALY_WEATHER!EXPLAIN_FEATURE_IMPORTANCE();

#### ステップ 5: カスタム — prediction_interval を変更して感度を比較

`prediction_interval` は「どの程度の信頼区間で異常と判定するか」を制御するパラメータです。

- **0.95**（デフォルト）: 95% 信頼区間 → やや敏感（異常を多く検知）
- **0.99**: 99% 信頼区間 → より厳格（明らかな異常だけ検知）

両方の結果を比較して、**感度 vs 特異度のトレードオフ** を体験しましょう。

In [ ]:
-- 気象データ付きモデルで異常検知（prediction_interval = 0.95）
CREATE OR REPLACE TABLE TELECOM_AI_HANDSON.ANALYTICS.ANOMALY_WEATHER_95 AS
SELECT * FROM TABLE(TELECOM_AI_HANDSON.ANALYTICS.NQ_ANOMALY_WEATHER!DETECT_ANOMALIES(
    INPUT_DATA => TABLE(TELECOM_AI_HANDSON.ANALYTICS.NQ_TEST_WITH_WEATHER),
    TIMESTAMP_COLNAME => 'TS',
    TARGET_COLNAME => 'CALL_DROP_RATE',
    CONFIG_OBJECT => {'prediction_interval': 0.95}
));

-- 同じモデルで prediction_interval = 0.99 で再検知
CREATE OR REPLACE TABLE TELECOM_AI_HANDSON.ANALYTICS.ANOMALY_WEATHER_99 AS
SELECT * FROM TABLE(TELECOM_AI_HANDSON.ANALYTICS.NQ_ANOMALY_WEATHER!DETECT_ANOMALIES(
    INPUT_DATA => TABLE(TELECOM_AI_HANDSON.ANALYTICS.NQ_TEST_WITH_WEATHER),
    TIMESTAMP_COLNAME => 'TS',
    TARGET_COLNAME => 'CALL_DROP_RATE',
    CONFIG_OBJECT => {'prediction_interval': 0.99}
));

-- 感度比較: 各 interval で検出された異常数を比較
SELECT
    '0.95 (デフォルト)' AS PREDICTION_INTERVAL,
    COUNT(*) AS TOTAL_DAYS,
    SUM(CASE WHEN IS_ANOMALY THEN 1 ELSE 0 END) AS ANOMALY_DAYS,
    ROUND(SUM(CASE WHEN IS_ANOMALY THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) AS ANOMALY_PCT
FROM TELECOM_AI_HANDSON.ANALYTICS.ANOMALY_WEATHER_95
UNION ALL
SELECT
    '0.99 (厳格)',
    COUNT(*),
    SUM(CASE WHEN IS_ANOMALY THEN 1 ELSE 0 END),
    ROUND(SUM(CASE WHEN IS_ANOMALY THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1)
FROM TELECOM_AI_HANDSON.ANALYTICS.ANOMALY_WEATHER_99;

### パート B: トラフィック量の将来予測（FORECAST）

設備が **逼迫** しているエリアのトラフィック量を予測し、**あと何日で設備容量を超過するか** をシミュレーションします。  
ここでも気象データを外因変数として追加し、予測精度の改善を体験します。

#### ステップ 1: 設備逼迫エリアの確認とトレーニングデータ準備

In [ ]:
-- 設備逼迫エリアを確認
SELECT AREA_ID, AREA_NAME, REGION_NAME, CURRENT_LOAD_PCT, MAX_CAPACITY_GBPS, EQUIPMENT_STATUS
FROM TELECOM_AI_HANDSON.ANALYTICS.EQUIPMENT_STATUS
WHERE EQUIPMENT_STATUS = '逼迫'
ORDER BY CURRENT_LOAD_PCT DESC;

#### ステップ 2: 気温を外因変数にしたトラフィック予測モデルの訓練

東京都心南部エリア（負荷率92.2%で最も逼迫）を対象に、気温を外因変数として FORECAST モデルを訓練します。  
夏場の暑い日はエアコン下での動画視聴が増え、冬場はイベント時のトラフィックが集中するなど、**気温とトラフィックには相関** があります。

In [ ]:
-- 訓練データ: 東京都心南部の日次トラフィック + 気温（2022〜2025年）
CREATE OR REPLACE VIEW TELECOM_AI_HANDSON.ANALYTICS.TRAFFIC_TRAIN AS
SELECT
    nq.MEASURE_DATE::TIMESTAMP_NTZ AS TS,
    nq.TOTAL_TRAFFIC_GB::FLOAT AS TOTAL_TRAFFIC_GB,
    w.AVG_TEMPERATURE::FLOAT AS AVG_TEMPERATURE
FROM TELECOM_AI_HANDSON.ANALYTICS.NETWORK_QUALITY nq
JOIN TELECOM_AI_HANDSON.ANALYTICS.DAILY_WEATHER w
    ON nq.MEASURE_DATE = w.WEATHER_DATE
WHERE nq.AREA_ID = 'AREA_KT_002'  -- 東京都心南部エリア
ORDER BY nq.MEASURE_DATE;

-- FORECAST モデルの訓練（気温を外因変数として使用）
CREATE OR REPLACE SNOWFLAKE.ML.FORECAST TELECOM_AI_HANDSON.ANALYTICS.TRAFFIC_FORECAST_MODEL(
    INPUT_DATA => TABLE(TELECOM_AI_HANDSON.ANALYTICS.TRAFFIC_TRAIN),
    TIMESTAMP_COLNAME => 'TS',
    TARGET_COLNAME => 'TOTAL_TRAFFIC_GB'
);

-- 特徴量の重要度を確認（気温がどれだけ寄与しているか）
CALL TELECOM_AI_HANDSON.ANALYTICS.TRAFFIC_FORECAST_MODEL!EXPLAIN_FEATURE_IMPORTANCE();

#### ステップ 3: 30日先のトラフィック予測と設備容量超過シミュレーション

外因変数（気温）を使ったモデルなので、予測期間の気温も渡す必要があります。  
ここでは **前年同時期の気温データ** を将来の気温推定値として使う実践的テクニックを紹介します。

In [ ]:
-- 将来30日分の気温データを前年同時期から生成
-- FLOAT カラムを明示定義（FORECAST の外因変数は非NULL の FLOAT 型が必要）
CREATE OR REPLACE TEMP TABLE TELECOM_AI_HANDSON.ANALYTICS.FUTURE_WEATHER (
    TS TIMESTAMP_NTZ,
    AVG_TEMPERATURE FLOAT
);

-- 前年同日の気温を取得。前年データがなければ前後1日で補完
INSERT INTO TELECOM_AI_HANDSON.ANALYTICS.FUTURE_WEATHER
WITH base AS (
    SELECT DATEADD(DAY, 1, MAX(MEASURE_DATE)) AS START_DATE
    FROM TELECOM_AI_HANDSON.ANALYTICS.NETWORK_QUALITY
),
future_dates AS (
    SELECT
        DATEADD(DAY, SEQ4(), (SELECT START_DATE FROM base))::TIMESTAMP_NTZ AS TS,
        DATEADD(DAY, SEQ4(), (SELECT START_DATE FROM base))::DATE          AS FUTURE_DATE
    FROM TABLE(GENERATOR(ROWCOUNT => 30))
)
SELECT
    f.TS,
    COALESCE(
        w.AVG_TEMPERATURE,
        -- 前年同日がなければ前年±1日の平均で補完
        (SELECT AVG(w2.AVG_TEMPERATURE)
         FROM TELECOM_AI_HANDSON.ANALYTICS.DAILY_WEATHER w2
         WHERE w2.WEATHER_DATE BETWEEN DATEADD(YEAR, -1, DATEADD(DAY, -1, f.FUTURE_DATE))
                                    AND DATEADD(YEAR, -1, DATEADD(DAY,  1, f.FUTURE_DATE)))
    )::FLOAT AS AVG_TEMPERATURE
FROM future_dates f
LEFT JOIN TELECOM_AI_HANDSON.ANALYTICS.DAILY_WEATHER w
    ON w.WEATHER_DATE = DATEADD(YEAR, -1, f.FUTURE_DATE)
ORDER BY f.TS;

-- NULL が残っていないことを確認
SELECT COUNT(*) AS TOTAL, COUNT(AVG_TEMPERATURE) AS NON_NULL
FROM TELECOM_AI_HANDSON.ANALYTICS.FUTURE_WEATHER;

-- 30日先のトラフィック予測を実行
CREATE OR REPLACE TABLE TELECOM_AI_HANDSON.ANALYTICS.TRAFFIC_FORECAST_RESULTS AS
SELECT * FROM TABLE(TELECOM_AI_HANDSON.ANALYTICS.TRAFFIC_FORECAST_MODEL!FORECAST(
    INPUT_DATA => TABLE(TELECOM_AI_HANDSON.ANALYTICS.FUTURE_WEATHER),
    TIMESTAMP_COLNAME => 'TS'
));

In [ ]:
-- 予測結果と設備容量を比較: いつ容量超過するか？
-- 東京都心南部エリアの最大容量: 140.1 Gbps → 日次換算 (概算)
WITH capacity AS (
    SELECT MAX_CAPACITY_GBPS * 1000 AS DAILY_CAPACITY_GB  -- Gbps を概算で日次GBに換算
    FROM TELECOM_AI_HANDSON.ANALYTICS.EQUIPMENT_STATUS
    WHERE AREA_ID = 'AREA_KT_002'
)
SELECT
    f.TS::DATE AS FORECAST_DATE,
    ROUND(f.FORECAST, 1) AS PREDICTED_TRAFFIC_GB,
    ROUND(f.UPPER_BOUND, 1) AS UPPER_BOUND_GB,
    ROUND(c.DAILY_CAPACITY_GB, 1) AS CAPACITY_GB,
    ROUND(f.FORECAST / c.DAILY_CAPACITY_GB * 100, 1) AS LOAD_PCT,
    CASE
        WHEN f.UPPER_BOUND > c.DAILY_CAPACITY_GB THEN '⚠ 超過リスクあり'
        WHEN f.FORECAST > c.DAILY_CAPACITY_GB * 0.9 THEN '△ 注意'
        ELSE '○ 正常'
    END AS STATUS
FROM TELECOM_AI_HANDSON.ANALYTICS.TRAFFIC_FORECAST_RESULTS f
CROSS JOIN capacity c
ORDER BY f.TS;

### パート C: 品質劣化の要因分析（TOP_INSIGHTS）

パート A で切断率の異常日が特定できました。  
次は **「なぜその日に品質が劣化したのか？」** を自動分析します。

`TOP_INSIGHTS` は、**2つのグループ（異常期間 vs 通常期間）のメトリクスの差を分析し、どのディメンション（地方、エリアタイプなど）が最も差に寄与しているか** を決定木で自動特定します。

In [ ]:
-- TOP_INSIGHTS で品質劣化の要因を分析
-- 異常日 vs 通常日を label で区別し、地方・エリアごとの切断率差を自動特定
CREATE OR REPLACE SNOWFLAKE.ML.TOP_INSIGHTS TELECOM_AI_HANDSON.ANALYTICS.NQ_INSIGHTS_MODEL();

CREATE OR REPLACE VIEW TELECOM_AI_HANDSON.ANALYTICS.NQ_INSIGHTS_INPUT AS
SELECT
    nq.REGION_NAME,
    nq.AREA_NAME,
    nq.CALL_DROP_RATE::FLOAT AS CALL_DROP_RATE,
    nq.PACKET_LOSS_RATE::FLOAT AS PACKET_LOSS_RATE,
    w.AVG_RAINFALL::FLOAT AS AVG_RAINFALL,
    w.AVG_WIND_SPEED::FLOAT AS AVG_WIND_SPEED,
    -- 異常日かどうかのラベル（パートAの結果を利用）
    CASE
        WHEN nq.MEASURE_DATE IN (SELECT TS::DATE FROM TELECOM_AI_HANDSON.ANALYTICS.ANOMALY_WEATHER_95 WHERE IS_ANOMALY = TRUE)
        THEN TRUE ELSE FALSE
    END AS IS_ANOMALY_DAY
FROM TELECOM_AI_HANDSON.ANALYTICS.NETWORK_QUALITY nq
JOIN TELECOM_AI_HANDSON.ANALYTICS.DAILY_WEATHER w
    ON nq.MEASURE_DATE = w.WEATHER_DATE
WHERE nq.MEASURE_DATE >= '2025-01-01';

CALL TELECOM_AI_HANDSON.ANALYTICS.NQ_INSIGHTS_MODEL!GET_DRIVERS(
    INPUT_DATA => TABLE(TELECOM_AI_HANDSON.ANALYTICS.NQ_INSIGHTS_INPUT),
    LABEL_COLNAME => 'IS_ANOMALY_DAY',
    METRIC_COLNAME => 'CALL_DROP_RATE'
);

### Step 3 まとめ

SQL だけで以下の ML パイプラインを構築できました：

| 機能 | ML Function | カスタム要素 |
|------|------------|------------|
| 異常検知 | `ANOMALY_DETECTION` | 気象データを外因変数に追加、prediction_interval で感度調整 |
| 将来予測 | `FORECAST` | 気温を外因変数に追加、前年データで将来値を生成 |
| 要因分析 | `TOP_INSIGHTS` | 異常日 vs 通常日で地方・気象要因の寄与度を自動特定 |

> **発展**: この異常検知を Snowflake の **Task + Alert** と組み合わせると、「切断率が異常値を超えたら自動でメール通知」という運用パイプラインも構築できます。

> **応用編**: Python で独自モデル（IsolationForest / XGBoost）を構築したい方は **Step 8** に進んでください。

---
## Step 4: エリアの意味検索を可能にする（Cortex Search）

Step 1 で作った `AI_AREA_MASTER` に対して、**Cortex Search Service** を作成します。

### Cortex Search とは？

テキストの **意味** を理解して検索するサービスです。  
キーワードの完全一致ではなく、「こういう意味のものを探して」という検索ができます。

例えば「トラフィックが多い都市部のエリア」と聞くと、東京都心や大阪梅田などのエリアがヒットします。

### 仕組み

エリア名・地方名・エリアタイプ・AI説明文を **1つのテキストに結合** し、  
Snowflake の Embedding モデル（`snowflake-arctic-embed-l-v2.0`）でベクトル化して検索可能にします。

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE TELECOM_AI_HANDSON.ANALYTICS.AREA_SEARCH_SERVICE
    ON SEARCH_TEXT
    ATTRIBUTES AREA_ID, AREA_NAME, REGION_NAME, AREA_TYPE, AREA_DESCRIPTION
    WAREHOUSE = COMPUTE_WH
    TARGET_LAG = '1 day'
    EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
AS (
    SELECT
        AREA_ID,
        AREA_NAME,
        REGION_NAME,
        REGION_CODE,
        AREA_TYPE,
        AREA_DESCRIPTION,
        CONCAT(
            COALESCE(AREA_NAME, ''), ' ',
            COALESCE(REGION_NAME, ''), ' ',
            COALESCE(AREA_TYPE, ''), ' ',
            COALESCE(AREA_DESCRIPTION, '')
        ) AS SEARCH_TEXT
    FROM TELECOM_AI_HANDSON.ANALYTICS.AI_AREA_MASTER
);

---
## Step 5: データの意味を定義する（セマンティックビュー）

ここがこのハンズオンの **キモ** です。

Cortex Analyst に「通信速度を教えて」と聞いたとき、どのテーブルの・どのカラムを・どう集計すればよいかを  
AI が理解できるように、**データの意味（セマンティクス）** を定義します。

### セマンティックビューで定義すること

| 定義項目 | 何を定義する？ | 具体例 |
|----------|---------------|--------|
| **テーブル** | 分析対象のテーブルとその説明 | NETWORK_QUALITY, EQUIPMENT_STATUS, WEATHER |
| **リレーションシップ** | テーブル間の結合条件 | 計測日 = 気象観測日 |
| **ファクト** | 集計対象の数値カラム | 通信速度、遅延、トラフィック量、気温 |
| **ディメンション** | 分析の切り口となる属性 | エリア名、地方、日付 |
| **メトリクス** | よく使う集計パターン | 平均ダウンロード速度＝AVG(下り速度) |
| **シノニム** | 日本語の別名・同義語 | 「通信速度」「下り速度」「ダウンロード速度」 |

### 3つのテーブルの関係

```
              AREA_ID              MEASURE_DATE = WEATHER_DATE
NETWORK_QUALITY ───────▶ EQUIPMENT_STATUS    NETWORK_QUALITY ────────────────▶ WEATHER
(通信品質)                (設備状況)           (通信品質)                        (気象)
```

> **ポイント**: シノニムを豊富に定義しておくと、「通信速度」でも「ダウンロード速度」でも「下り」でも同じカラムを参照してくれます。

In [ ]:
CREATE OR REPLACE SEMANTIC VIEW TELECOM_AI_HANDSON.ANALYTICS.TELECOM_ANALYST

    -- ========================================
    -- テーブル定義
    -- ========================================
    tables (
        NETWORK_QUALITY AS TELECOM_AI_HANDSON.ANALYTICS.NETWORK_QUALITY
            PRIMARY KEY (AREA_ID, MEASURE_DATE)
            WITH SYNONYMS = ('通信品質', 'ネットワーク品質', '品質データ', '通信データ', 'NW品質', '通信速度データ', 'KPI')
            COMMENT = '日次ネットワーク品質データ。2022-01-01〜2026-02-24のエリア別日次通信速度・遅延・接続成功率・トラフィック量を含む。',

        EQUIPMENT_STATUS AS TELECOM_AI_HANDSON.ANALYTICS.EQUIPMENT_STATUS
            PRIMARY KEY (AREA_ID)
            WITH SYNONYMS = ('設備', '設備状況', '設備データ', '基地局', 'キャパシティ', '設備キャパシティ', '負荷')
            COMMENT = '設備キャパシティスナップショット（2026-02-24時点）。エリア別の基地局数・最大容量・負荷率・設備ステータスを含む。',

        WEATHER AS TELECOM_AI_HANDSON.ANALYTICS.DAILY_WEATHER
            PRIMARY KEY (WEATHER_DATE)
            WITH SYNONYMS = ('気象', '天気', '気象データ', '天気データ', '気温', '降水量', '天候', '気象庁', '気象情報')
            COMMENT = '全国日次平均気象データ（2015年〜）。気象庁の全観測所データを日別に集約した全国平均の気温・降水量・湿度・日照時間・降雪量・風速。'
    )

    -- ========================================
    -- リレーションシップ定義
    -- ========================================
    relationships (
        NQ_EQUIPMENT AS NETWORK_QUALITY(AREA_ID) REFERENCES EQUIPMENT_STATUS(AREA_ID),
        NQ_WEATHER AS NETWORK_QUALITY(MEASURE_DATE) REFERENCES WEATHER(WEATHER_DATE)
    )

    -- ========================================
    -- ファクト定義（数値データ）
    -- ========================================
    facts (
        -- 通信品質ファクト
        NETWORK_QUALITY.download_speed_fact AS AVG_DOWNLOAD_MBPS
            COMMENT = '平均ダウンロード速度（Mbps）。下り通信速度。',
        NETWORK_QUALITY.upload_speed_fact AS AVG_UPLOAD_MBPS
            COMMENT = '平均アップロード速度（Mbps）。上り通信速度。',
        NETWORK_QUALITY.latency_fact AS AVG_LATENCY_MS
            COMMENT = '平均遅延（ミリ秒）。レイテンシ。',
        NETWORK_QUALITY.connection_success_fact AS CONNECTION_SUCCESS_RATE
            COMMENT = '接続成功率（%）。100%に近いほど良好。',
        NETWORK_QUALITY.call_drop_fact AS CALL_DROP_RATE
            COMMENT = '切断率（%）。0%に近いほど良好。',
        NETWORK_QUALITY.packet_loss_fact AS PACKET_LOSS_RATE
            COMMENT = 'パケットロス率（%）。0%に近いほど良好。',
        NETWORK_QUALITY.traffic_fact AS TOTAL_TRAFFIC_GB
            COMMENT = '日次総トラフィック量（GB）',

        -- 設備ファクト
        EQUIPMENT_STATUS.base_station_count_fact AS BASE_STATION_COUNT
            COMMENT = '基地局数（局）',
        EQUIPMENT_STATUS.max_capacity_fact AS MAX_CAPACITY_GBPS
            COMMENT = '最大通信容量（Gbps）',
        EQUIPMENT_STATUS.current_load_fact AS CURRENT_LOAD_PCT
            COMMENT = '現在の設備負荷率（%）。100%に近いほど逼迫。',

        -- 気象ファクト
        WEATHER.avg_temperature_fact AS AVG_TEMPERATURE
            COMMENT = '全国日平均気温（℃）。全観測所の平均。',
        WEATHER.max_temperature_fact AS MAX_TEMPERATURE
            COMMENT = '全国日最高気温（℃）。全観測所の最大値。',
        WEATHER.min_temperature_fact AS MIN_TEMPERATURE
            COMMENT = '全国日最低気温（℃）。全観測所の最小値。',
        WEATHER.avg_rainfall_fact AS AVG_RAINFALL
            COMMENT = '全国日平均降水量（mm）',
        WEATHER.avg_humidity_fact AS AVG_HUMIDITY
            COMMENT = '全国日平均湿度（%）',
        WEATHER.avg_sunlight_hours_fact AS AVG_SUNLIGHT_HOURS
            COMMENT = '全国日平均日照時間（時間）',
        WEATHER.total_snowfall_fact AS TOTAL_SNOWFALL
            COMMENT = '全国日降雪量合計（cm）',
        WEATHER.avg_wind_speed_fact AS AVG_WIND_SPEED
            COMMENT = '全国日平均風速（km/h）'
    )

    -- ========================================
    -- ディメンション定義（属性データ）
    -- ========================================
    dimensions (
        -- ========== NETWORK_QUALITY ディメンション ==========
        NETWORK_QUALITY.area_id AS AREA_ID
            WITH SYNONYMS = ('エリアID', 'エリアコード', '基地局エリア', 'エリア識別子')
            COMMENT = 'エリア識別コード（例：AREA_KT_001）',
        NETWORK_QUALITY.area_name AS AREA_NAME
            WITH SYNONYMS = ('エリア名', 'エリア', '地域名', '地区名', '基地局エリア名')
            COMMENT = '通信エリア名。東京都心北部エリア、大阪梅田エリアなどを含む。',
        NETWORK_QUALITY.region_name AS REGION_NAME
            WITH SYNONYMS = ('地方', '地方名', 'リージョン', '地域')
            COMMENT = '地方名（関東、近畿、東海、九州）',
        NETWORK_QUALITY.measure_date AS MEASURE_DATE
            WITH SYNONYMS = ('計測日', '測定日', '日付', '対象日')
            COMMENT = '通信品質を計測した日付（DATE型、2022-01-01〜2026-02-24）',
        NETWORK_QUALITY.measure_year AS YEAR(MEASURE_DATE)
            WITH SYNONYMS = ('計測年', '年', '対象年')
            COMMENT = '計測年（2022〜2026）',
        NETWORK_QUALITY.measure_month AS MONTH(MEASURE_DATE)
            WITH SYNONYMS = ('計測月', '月', '対象月')
            COMMENT = '計測月（1〜12）',

        -- ========== EQUIPMENT_STATUS ディメンション ==========
        EQUIPMENT_STATUS.area_id AS AREA_ID
            WITH SYNONYMS = ('エリアID', 'エリアコード')
            COMMENT = 'エリア識別コード',
        EQUIPMENT_STATUS.area_name AS AREA_NAME
            WITH SYNONYMS = ('エリア名', 'エリア')
            COMMENT = 'エリア名',
        EQUIPMENT_STATUS.region_name AS REGION_NAME
            WITH SYNONYMS = ('地方', '地方名')
            COMMENT = '地方名',
        EQUIPMENT_STATUS.snapshot_date AS SNAPSHOT_DATE
            WITH SYNONYMS = ('スナップショット日', '基準日', '設備確認日')
            COMMENT = '設備データの取得日（2026-02-24）',
        EQUIPMENT_STATUS.equipment_status AS EQUIPMENT_STATUS
            WITH SYNONYMS = ('設備ステータス', '設備状態', '設備区分', 'ステータス', '負荷状態')
            COMMENT = '設備の負荷状態を表すステータス（正常 / 注意 / 逼迫）',

        -- ========== WEATHER ディメンション ==========
        WEATHER.weather_date AS WEATHER_DATE
            WITH SYNONYMS = ('観測日', '気象日', '天気の日付', '気象観測日')
            COMMENT = '気象データの観測日（DATE型、2015年〜）。NETWORK_QUALITYのMEASURE_DATEとリレーションで紐づく。',
        WEATHER.weather_year AS YEAR(WEATHER_DATE)
            WITH SYNONYMS = ('観測年', '気象年')
            COMMENT = '気象データの観測年',
        WEATHER.weather_month AS MONTH(WEATHER_DATE)
            WITH SYNONYMS = ('観測月', '気象月')
            COMMENT = '気象データの観測月（1〜12）'
    )

    -- ========================================
    -- メトリクス定義（集計関数）
    -- ========================================
    metrics (
        -- 通信品質メトリクス
        NETWORK_QUALITY.avg_download_speed AS AVG(NETWORK_QUALITY.download_speed_fact)
            WITH SYNONYMS = ('平均ダウンロード速度', '平均下り速度', '下り平均', '平均通信速度')
            COMMENT = 'ダウンロード速度の平均（Mbps）',
        NETWORK_QUALITY.avg_upload_speed AS AVG(NETWORK_QUALITY.upload_speed_fact)
            WITH SYNONYMS = ('平均アップロード速度', '平均上り速度', '上り平均')
            COMMENT = 'アップロード速度の平均（Mbps）',
        NETWORK_QUALITY.avg_latency AS AVG(NETWORK_QUALITY.latency_fact)
            WITH SYNONYMS = ('平均遅延', '平均レイテンシ', 'レイテンシ平均')
            COMMENT = '遅延の平均（ms）',
        NETWORK_QUALITY.avg_connection_success AS AVG(NETWORK_QUALITY.connection_success_fact)
            WITH SYNONYMS = ('平均接続成功率', '接続成功率平均', '接続率')
            COMMENT = '接続成功率の平均（%）',
        NETWORK_QUALITY.avg_call_drop AS AVG(NETWORK_QUALITY.call_drop_fact)
            WITH SYNONYMS = ('平均切断率', '切断率平均', '通話切断率')
            COMMENT = '切断率の平均（%）',
        NETWORK_QUALITY.avg_packet_loss AS AVG(NETWORK_QUALITY.packet_loss_fact)
            WITH SYNONYMS = ('平均パケットロス率', 'パケロス平均', 'ロス率')
            COMMENT = 'パケットロス率の平均（%）',
        NETWORK_QUALITY.total_traffic AS SUM(NETWORK_QUALITY.traffic_fact)
            WITH SYNONYMS = ('総トラフィック', '合計トラフィック', 'トラフィック合計', '総通信量')
            COMMENT = 'トラフィック量の合計（GB）',
        NETWORK_QUALITY.avg_daily_traffic AS AVG(NETWORK_QUALITY.traffic_fact)
            WITH SYNONYMS = ('日平均トラフィック', '日次平均トラフィック', '平均日次通信量')
            COMMENT = '日次トラフィック量の平均（GB）',

        -- 設備メトリクス
        EQUIPMENT_STATUS.total_base_stations AS SUM(EQUIPMENT_STATUS.base_station_count_fact)
            WITH SYNONYMS = ('総基地局数', '合計基地局数', '基地局数合計')
            COMMENT = '基地局数の合計（局）',
        EQUIPMENT_STATUS.total_capacity AS SUM(EQUIPMENT_STATUS.max_capacity_fact)
            WITH SYNONYMS = ('総容量', '合計容量', '最大容量合計')
            COMMENT = '最大通信容量の合計（Gbps）',
        EQUIPMENT_STATUS.avg_load AS AVG(EQUIPMENT_STATUS.current_load_fact)
            WITH SYNONYMS = ('平均負荷率', '負荷率平均', '平均設備負荷')
            COMMENT = '設備負荷率の平均（%）',

        -- 気象メトリクス
        WEATHER.period_avg_temperature AS AVG(WEATHER.avg_temperature_fact)
            WITH SYNONYMS = ('平均気温', '気温平均', '平均温度')
            COMMENT = '期間内の全国平均気温（℃）',
        WEATHER.period_max_temperature AS MAX(WEATHER.max_temperature_fact)
            WITH SYNONYMS = ('最高気温', '最高温度', '期間最高気温')
            COMMENT = '期間中の全国最高気温（℃）',
        WEATHER.period_min_temperature AS MIN(WEATHER.min_temperature_fact)
            WITH SYNONYMS = ('最低気温', '最低温度', '期間最低気温')
            COMMENT = '期間中の全国最低気温（℃）',
        WEATHER.period_avg_rainfall AS AVG(WEATHER.avg_rainfall_fact)
            WITH SYNONYMS = ('平均降水量', '降水量平均')
            COMMENT = '期間内の全国平均降水量（mm）',
        WEATHER.period_avg_humidity AS AVG(WEATHER.avg_humidity_fact)
            WITH SYNONYMS = ('平均湿度', '湿度平均')
            COMMENT = '期間内の全国平均湿度（%）',
        WEATHER.period_avg_sunlight AS AVG(WEATHER.avg_sunlight_hours_fact)
            WITH SYNONYMS = ('平均日照時間', '日照時間平均')
            COMMENT = '期間内の全国平均日照時間（時間）',
        WEATHER.period_total_snowfall AS SUM(WEATHER.total_snowfall_fact)
            WITH SYNONYMS = ('総降雪量', '合計降雪量', '累積降雪量')
            COMMENT = '期間内の全国降雪量合計（cm）',
        WEATHER.period_avg_wind_speed AS AVG(WEATHER.avg_wind_speed_fact)
            WITH SYNONYMS = ('平均風速', '風速平均')
            COMMENT = '期間内の全国平均風速（km/h）'
    )

    COMMENT = '携帯電話通信会社のネットワーク品質・設備・気象分析のためのセマンティックビュー。関東・近畿・東海・九州の50エリアの通信品質データを含む。2022年〜の日次通信品質データ（下り/上り速度、遅延、接続成功率、切断率、パケットロス率、トラフィック量）、2026-02-24時点の設備キャパシティスナップショット（基地局数、最大容量、負荷率）、および気象庁の全国日次平均気象データ（気温・降水量・湿度・日照時間・降雪量等）を統合。計測日と気象観測日がリレーションで紐づき、天候と通信品質の相関分析が可能。';

### セマンティックビューの確認

正しく定義できたか確認しましょう。テーブル・ファクト・ディメンション・メトリクスの一覧が表示されます。

In [ ]:
DESCRIBE SEMANTIC VIEW TELECOM_ANALYST;

---
## Step 6: AI エージェントを組み立てる（Cortex Agent）

いよいよエージェントを組み立てます。  
ここまで作ってきた **Cortex Search** と **Cortex Analyst** を組み合わせて、  
1つの **AI エージェント** にまとめます。

### エージェントの仕組み

```
ユーザーの質問
     │
     ▼
┌─────────────────────────┐
│   Telecom Agent        │
│  （オーケストレーター）   │
│                        │
│  質問の意図を判断して      │
│  適切なツールを選択        │
└────┬──────────┬─────────┘
     │          │
     ▼          ▼
┌─────────┐ ┌──────────────┐
│ Cortex  │ │   Cortex     │
│ Search  │ │  Analyst     │
│         │ │              │
│エリア情報│  │通信品質・設備・│
│の意味検索│ │気象の数値分析  │
└─────────┘ └──────────────┘
```

### ツールの使い分け

| 質問の種類 | 使われるツール | 例 |
|-----------|---------------|----|
| エリアの特徴・情報を知りたい | **Cortex Search** | 「関東のエリア一覧を教えて」 |
| 数値の集計・分析をしたい | **Cortex Analyst** | 「2025年の月別通信速度は？」 |
| 両方が必要 | **Search → Analyst** | 「逼迫エリアのトラフィック推移は？」 |

In [ ]:
CREATE OR REPLACE AGENT TELECOM_AI_HANDSON.ANALYTICS.TELECOM_AGENT
    COMMENT = '携帯電話通信会社のネットワーク品質・設備・気象分析とエリア検索を行うエージェント'
FROM SPECIFICATION $$
{
  "models": {
    "orchestration": ""
  },
  "instructions": {
    "response": "あなたは携帯電話通信会社のネットワーク品質アナリストです。ユーザーからの質問に対して、通信品質データ（下り/上り速度、遅延、接続成功率、切断率、パケットロス率、トラフィック量）、設備キャパシティデータ（基地局数、最大容量、負荷率）、気象データを活用して回答してください。数値データは適切にフォーマットして表示してください（例：速度はMbps表記、トラフィックはGB表記、比率は%表記）。エリア名は正確に記載してください。分析結果には根拠となるデータを含めてください。日本語で丁寧に回答してください。可能であれば視覚化を提供してください。時系列のトレンドは線グラフをデフォルトとし、エリア比較は棒グラフを使用してください。天候と通信品質の関係（雨天時の品質低下、気温とトラフィック量の相関など）にも注意して分析してください。",
    "orchestration": "エリアの詳細情報（エリア名、地方、エリアタイプ、AI説明文など）を検索する場合はcortex searchを使用し、通信品質・設備・気象の数値分析にはcortex analystを使用してください。複合的な質問の場合は、まずcortex searchでエリアを特定し、その結果をcortex analystに渡して分析してください。気温や天候と通信品質の関係を聞かれた場合は、cortex analystでネットワーク品質データと気象データを組み合わせて分析してください。",
    "sample_questions": [
      {
        "question": "2025年の東京都心エリアの下り通信速度推移を月別で可視化してください。気温との相関も見たいです。"
      },
      {
        "question": "トラフィック量が最も多い月はいつですか？どのエリアが特にトラフィックが集中しますか？"
      },
      {
        "question": "現在設備が逼迫しているエリアを教えてください。それぞれのエリアの過去のトラフィック推移も見せてください。"
      },
      {
        "question": "関東地方のエリア一覧とそれぞれの特徴を教えてください。"
      },
      {
        "question": "2024年と2025年でトラフィックが大きく増加したエリアトップ5を教えてください。"
      }
    ]
  },
  "tools": [
    {
      "tool_spec": {
        "type": "cortex_analyst_text_to_sql",
        "name": "TELECOM_ANALYST",
        "description": "携帯電話通信会社のネットワーク品質データ、設備キャパシティデータ、全国日次気象データをクエリして分析します。下り/上り通信速度、遅延、接続成功率、切断率、パケットロス率、トラフィック量の集計や、気温・降水量と通信品質の相関分析に使用します。2022年〜2026年2月の日次通信品質データ、2026年2月時点の設備キャパシティスナップショット、2015年〜の全国平均気象データを含みます。"
      }
    },
    {
      "tool_spec": {
        "type": "cortex_search",
        "name": "AREA_SEARCH",
        "description": "携帯電話通信エリアのマスタからエリア情報を検索します。エリア名、地方名、AIが分類したエリアタイプ（都心ビジネス、商業・繁華街、住宅地、観光・ウォーターフロント、郊外・地方）、AIが生成したエリア特性説明文などのテキスト検索に使用します。"
      }
    }
  ],
  "tool_resources": {
    "TELECOM_ANALYST": {
      "semantic_view": "TELECOM_AI_HANDSON.ANALYTICS.TELECOM_ANALYST"
    },
    "AREA_SEARCH": {
      "name": "TELECOM_AI_HANDSON.ANALYTICS.AREA_SEARCH_SERVICE",
      "max_results": 10
    }
  }
}
$$;

### エージェントの確認

In [ ]:
SHOW AGENTS IN SCHEMA TELECOM_AI_HANDSON.ANALYTICS;

---
## Step 7: 動かしてみよう！

エージェントが完成しました！Snowsight の **Cortex Agent UI** から以下のような質問を試してみましょう。

### 試してみたい質問例

**通信品質分析**
- 「2025年の東京都心エリアの下り通信速度推移を月別で可視化してください。気温との相関も見たいです。」
- 「2024年と2025年でトラフィックが大きく増加したエリアトップ5を教えてください。」

**天候×通信品質**
- 「降水量が多い日と少ない日で通信速度にどんな違いがありますか？」
- 「台風シーズン（8-9月）の通信品質への影響を分析してください。」

**設備管理**
- 「現在設備が逼迫しているエリアを教えてください。それぞれの過去のトラフィック推移も見せてください。」
- 「設備負荷率が注意レベルのエリアはどこですか？」

**エリア検索**
- 「関東地方のエリア一覧とそれぞれの特徴を教えてください。」
- 「観光地エリアの通信品質の傾向を教えてください。」

**Web検索**
- 「2026年に予定されている5G展開計画について教えてください。」
- 「最近の通信障害ニュースはありますか？」

---
## Step 8【応用編】: カスタムモデルで予兆検知（Snowpark ML / Container Runtime）

> **前提条件**: この Step は Snowflake Notebook の **Container Runtime** 上で実行します。  
> `setup_database.sql` の STEP 12 でコンピュートプールの作成と Notebook への割り当てが完了しています。  
> Snowsight の Notebook 設定で **Container Runtime が有効** になっていることを確認してください。  
> scikit-learn / XGBoost は Container Runtime に **プリインストール済み** のため、追加インストールは不要です。

### Step 3 との違い

| | Step 3（ML Functions） | Step 8（Snowpark ML） |
|---|---|---|
| **言語** | SQL のみ | Python（OSS ライブラリ） |
| **アルゴリズム** | Snowflake 組み込み（GBM / ARIMA） | 自由に選択（IsolationForest, XGBoost 等） |
| **カスタマイズ性** | 外因変数・感度パラメータ | ハイパーパラメータ・特徴量エンジニアリング全般 |
| **実行環境** | ウェアハウス | Container Runtime（コンピュートプール） |
| **モデル管理** | SQL オブジェクト | **Model Registry**（バージョン管理・メタデータ） |

### この Step でやること

1. **IsolationForest** で切断率の異常検知（教師なし）
2. **XGBRegressor** でトラフィック量の予測（教師あり）
3. 両モデルを **Model Registry** に登録し、推論を実行

In [ ]:
# Container Runtime にプリインストール済みのパッケージを確認
# scikit-learn と xgboost が含まれていることを確認します
!pip freeze | grep -E "scikit-learn|xgboost"

In [ ]:
# ============================================================
# データ取得: 通信品質 + 気象データを Pandas に読み込む
# ============================================================
import pandas as pd
from snowflake.snowpark.context import get_active_session

session = get_active_session()

# 全国平均の日次通信品質 + 気象データを結合して取得
df = session.sql("""
    SELECT
        nq.MEASURE_DATE,
        AVG(nq.CALL_DROP_RATE)          AS CALL_DROP_RATE,
        AVG(nq.TOTAL_TRAFFIC_GB)        AS TOTAL_TRAFFIC_GB,
        AVG(nq.AVG_DOWNLOAD_MBPS)       AS AVG_DOWNLOAD_MBPS,
        AVG(nq.AVG_LATENCY_MS)          AS AVG_LATENCY_MS,
        AVG(nq.PACKET_LOSS_RATE)        AS PACKET_LOSS_RATE,
        AVG(w.AVG_TEMPERATURE)          AS AVG_TEMPERATURE_C,
        AVG(w.AVG_RAINFALL)             AS PRECIPITATION_MM,
        AVG(w.AVG_WIND_SPEED)           AS WIND_SPEED_KMH
    FROM TELECOM_AI_HANDSON.ANALYTICS.NETWORK_QUALITY nq
    LEFT JOIN TELECOM_AI_HANDSON.ANALYTICS.DAILY_WEATHER w
        ON nq.MEASURE_DATE = w.WEATHER_DATE
    GROUP BY nq.MEASURE_DATE
    ORDER BY nq.MEASURE_DATE
""").to_pandas()

# 日付型変換・欠損値除去
df["MEASURE_DATE"] = pd.to_datetime(df["MEASURE_DATE"])
df = df.dropna()

# 訓練 / テスト分割（2025年以降をテスト）
train = df[df["MEASURE_DATE"] < "2025-01-01"].copy()
test  = df[df["MEASURE_DATE"] >= "2025-01-01"].copy()

print(f"訓練データ: {len(train)} 日, テストデータ: {len(test)} 日")
print(f"期間: {train['MEASURE_DATE'].min().date()} ~ {test['MEASURE_DATE'].max().date()}")
train.head()

### パート A: IsolationForest で異常検知

**IsolationForest** は、教師なし学習の異常検知アルゴリズムです。  
データポイントを「孤立させるのに必要な分割回数」で異常度を測定します。  
異常なデータは少ない分割で孤立するため、**異常スコアが高く** なります。

#### Step 3 の ANOMALY_DETECTION との違い

| | ML Functions（Step 3） | IsolationForest（Step 8） |
|---|---|---|
| アルゴリズム | GBM（Gradient Boosting） | ランダムフォレストベースの孤立検知 |
| ハイパーパラメータ | `prediction_interval` のみ | `n_estimators`, `contamination`, `max_features` 等 |
| 特徴量 | 対象カラム + 外因変数 | 自由に設計（ラグ特徴量・移動平均等も可） |
| 出力 | IS_ANOMALY（True/False） | 異常スコア（連続値） + ラベル |

In [ ]:
# ============================================================
# IsolationForest: 訓練 → Model Registry に登録
# ============================================================
from sklearn.ensemble import IsolationForest
from snowflake.ml.registry import Registry

# --- 特徴量 ---
FEATURE_COLS = [
    "CALL_DROP_RATE", "AVG_DOWNLOAD_MBPS", "AVG_LATENCY_MS",
    "PACKET_LOSS_RATE", "PRECIPITATION_MM", "WIND_SPEED_KMH"
]

# --- モデル訓練 ---
# contamination: 異常と見なすデータの割合（5%）
# n_estimators: 決定木の本数（多いほど安定）
iso_model = IsolationForest(
    n_estimators=200,
    contamination=0.05,
    max_features=0.8,
    random_state=42
)
iso_model.fit(train[FEATURE_COLS])

# --- 訓練データでの異常検知結果を確認 ---
train_scores = iso_model.decision_function(train[FEATURE_COLS])
train_pred   = iso_model.predict(train[FEATURE_COLS])
print(f"訓練データの異常件数: {(train_pred == -1).sum()} / {len(train_pred)}")

# --- Model Registry に登録 ---
# target_platforms=["WAREHOUSE"] でウェアハウスからの推論を有効化
reg = Registry(session=session, database_name="TELECOM_AI_HANDSON", schema_name="ANALYTICS")

# 既存モデルがあれば削除して再登録（冪等性を確保）
try:
    reg.delete_model("NQ_ISOLATION_FOREST")
    print("既存モデル NQ_ISOLATION_FOREST を削除しました")
except Exception:
    pass

iso_mv = reg.log_model(
    model=iso_model,
    model_name="NQ_ISOLATION_FOREST",
    version_name="v1",
    conda_dependencies=["scikit-learn"],
    comment="IsolationForest: 通信品質の異常検知モデル（切断率+気象特徴量）",
    sample_input_data=train[FEATURE_COLS],
    target_platforms=["WAREHOUSE"],
    metrics={
        "n_estimators": 200,
        "contamination": 0.05,
        "train_anomaly_ratio": float((train_pred == -1).mean())
    }
)

print(f"✅ Model Registry に登録完了: NQ_ISOLATION_FOREST v1")

In [ ]:
# ============================================================
# IsolationForest: テストデータで推論 → 結果を確認
# ============================================================
import numpy as np

# --- テストデータで推論（ローカルの iso_model を直接使用）---
test_pred   = iso_model.predict(test[FEATURE_COLS])
test_scores = iso_model.decision_function(test[FEATURE_COLS])

test["ISO_ANOMALY"]       = (test_pred == -1)
test["ISO_ANOMALY_SCORE"] = test_scores

anomaly_days = test[test["ISO_ANOMALY"]]
print(f"テストデータの異常検知結果: {len(anomaly_days)} 日 / {len(test)} 日 を異常と判定")
print(f"異常日の切断率: 平均 {anomaly_days['CALL_DROP_RATE'].mean():.3f}, "
      f"最大 {anomaly_days['CALL_DROP_RATE'].max():.3f}")
print(f"正常日の切断率: 平均 {test[~test['ISO_ANOMALY']]['CALL_DROP_RATE'].mean():.3f}")

# --- Registry 経由の推論（mv.run はウェアハウスで実行）---
print("\n--- Registry 経由の推論（mv.run → Warehouse）---")
iso_mv_loaded = reg.get_model("NQ_ISOLATION_FOREST").version("v1")
# Snowpark DataFrame に変換して渡す
test_sp = session.create_dataframe(test[FEATURE_COLS].head(5))
registry_pred = iso_mv_loaded.run(test_sp, function_name="predict")
registry_pred.show()

# --- 異常日の上位10件を表示 ---
anomaly_days[["MEASURE_DATE", "CALL_DROP_RATE", "PRECIPITATION_MM",
              "WIND_SPEED_KMH", "ISO_ANOMALY_SCORE"]].sort_values(
    "ISO_ANOMALY_SCORE").head(10)

### パート B: XGBRegressor でトラフィック予測

**XGBRegressor**（XGBoost の回帰モデル）を使って、トラフィック量を予測します。  
Step 3 の `FORECAST` は時系列専用（ARIMA ベース）でしたが、ここでは **特徴量エンジニアリング** を自分で設計し、  
気象データ・曜日・月などの特徴量を組み合わせた勾配ブースティング回帰モデルを構築します。

#### カスタマイズのポイント

| カスタム要素 | 内容 |
|---|---|
| **ラグ特徴量** | 前日・3日前・7日前のトラフィック量 |
| **移動平均** | 直近7日間の移動平均 |
| **カレンダー特徴量** | 曜日（0-6）、月（1-12） |
| **気象特徴量** | 気温・降水量・風速 |
| **ハイパーパラメータ** | `n_estimators`, `max_depth`, `learning_rate` を調整 |

In [ ]:
# ============================================================
# XGBRegressor: 特徴量エンジニアリング → 訓練 → Registry 登録
# ============================================================
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# --- 特徴量エンジニアリング ---
def add_features(data):
    """ラグ特徴量・移動平均・カレンダー特徴量を追加"""
    d = data.copy()
    d = d.sort_values("MEASURE_DATE")
    # ラグ特徴量
    d["TRAFFIC_LAG_1"]  = d["TOTAL_TRAFFIC_GB"].shift(1)
    d["TRAFFIC_LAG_3"]  = d["TOTAL_TRAFFIC_GB"].shift(3)
    d["TRAFFIC_LAG_7"]  = d["TOTAL_TRAFFIC_GB"].shift(7)
    # 移動平均
    d["TRAFFIC_MA_7"]   = d["TOTAL_TRAFFIC_GB"].rolling(7).mean()
    # カレンダー特徴量
    d["DAY_OF_WEEK"]    = d["MEASURE_DATE"].dt.dayofweek
    d["MONTH"]          = d["MEASURE_DATE"].dt.month
    return d.dropna()

train_fe = add_features(train)
# テストデータ: 訓練データの末尾と連結してラグ特徴量を計算した後に分離
all_data_fe = add_features(pd.concat([train, test], ignore_index=True))
test_fe = all_data_fe[all_data_fe["MEASURE_DATE"] >= "2025-01-01"].copy()

XGB_FEATURES = [
    "TRAFFIC_LAG_1", "TRAFFIC_LAG_3", "TRAFFIC_LAG_7", "TRAFFIC_MA_7",
    "DAY_OF_WEEK", "MONTH",
    "AVG_TEMPERATURE_C", "PRECIPITATION_MM", "WIND_SPEED_KMH"
]
TARGET = "TOTAL_TRAFFIC_GB"

# --- モデル訓練 ---
xgb_model = XGBRegressor(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
xgb_model.fit(
    train_fe[XGB_FEATURES], train_fe[TARGET],
    eval_set=[(test_fe[XGB_FEATURES], test_fe[TARGET])],
    verbose=False
)

# --- 評価 ---
test_fe["XGB_PREDICTED"] = xgb_model.predict(test_fe[XGB_FEATURES])
mae  = mean_absolute_error(test_fe[TARGET], test_fe["XGB_PREDICTED"])
rmse = np.sqrt(mean_squared_error(test_fe[TARGET], test_fe["XGB_PREDICTED"]))
print(f"テストデータ評価: MAE = {mae:.2f} GB, RMSE = {rmse:.2f} GB")

# --- 特徴量の重要度 ---
importance = pd.DataFrame({
    "FEATURE": XGB_FEATURES,
    "IMPORTANCE": xgb_model.feature_importances_
}).sort_values("IMPORTANCE", ascending=False)
print("\n特徴量の重要度:")
print(importance.to_string(index=False))

# --- Model Registry に登録 ---
# 既存モデルがあれば削除して再登録（冪等性を確保）
try:
    reg.delete_model("NQ_TRAFFIC_XGBOOST")
    print("既存モデル NQ_TRAFFIC_XGBOOST を削除しました")
except Exception:
    pass

# target_platforms=["WAREHOUSE"] でウェアハウスからの推論を有効化
xgb_mv = reg.log_model(
    model=xgb_model,
    model_name="NQ_TRAFFIC_XGBOOST",
    version_name="v1",
    conda_dependencies=["xgboost"],
    comment="XGBRegressor: トラフィック予測（ラグ+気象+カレンダー特徴量）",
    sample_input_data=train_fe[XGB_FEATURES],
    target_platforms=["WAREHOUSE"],
    metrics={"MAE": float(mae), "RMSE": float(rmse)}
)

print(f"\n✅ Model Registry に登録完了: NQ_TRAFFIC_XGBOOST v1")

In [ ]:
# ============================================================
# Model Registry の確認: 登録したモデル一覧
# ============================================================

# Registry に登録されたモデル一覧
models = reg.show_models()
print("📦 登録済みモデル一覧:")
print(models[["name", "comment"]].to_string(index=False))

# 各モデルのバージョンとメトリクスを確認
for model_name in ["NQ_ISOLATION_FOREST", "NQ_TRAFFIC_XGBOOST"]:
    m = reg.get_model(model_name)
    mv = m.version("v1")
    print(f"\n--- {model_name} v1 ---")
    print(f"  メトリクス: {mv.show_metrics()}")

In [ ]:
# ============================================================
# Registry 経由の推論デモ: 直近30日の異常検知
# ============================================================

# 直近30日のデータを取得
recent_df = session.sql("""
    SELECT
        nq.MEASURE_DATE,
        AVG(nq.CALL_DROP_RATE)          AS CALL_DROP_RATE,
        AVG(nq.AVG_DOWNLOAD_MBPS)       AS AVG_DOWNLOAD_MBPS,
        AVG(nq.AVG_LATENCY_MS)          AS AVG_LATENCY_MS,
        AVG(nq.PACKET_LOSS_RATE)        AS PACKET_LOSS_RATE,
        AVG(w.AVG_RAINFALL)             AS PRECIPITATION_MM,
        AVG(w.AVG_WIND_SPEED)           AS WIND_SPEED_KMH
    FROM TELECOM_AI_HANDSON.ANALYTICS.NETWORK_QUALITY nq
    LEFT JOIN TELECOM_AI_HANDSON.ANALYTICS.DAILY_WEATHER w
        ON nq.MEASURE_DATE = w.WEATHER_DATE
    WHERE nq.MEASURE_DATE >= DATEADD('day', -30, CURRENT_DATE())
    GROUP BY nq.MEASURE_DATE
    ORDER BY nq.MEASURE_DATE DESC
""").to_pandas()
recent_df = recent_df.dropna()

# --- ローカルの iso_model で推論 ---
recent_pred   = iso_model.predict(recent_df[FEATURE_COLS])
recent_scores = iso_model.decision_function(recent_df[FEATURE_COLS])

recent_df["IS_ANOMALY"]     = (recent_pred == -1)
recent_df["ANOMALY_SCORE"]  = recent_scores

anomaly_count = recent_df["IS_ANOMALY"].sum()
print(f"直近30日の異常検知結果: {anomaly_count} 日を異常と判定")

# --- Registry 経由の推論（mv.run → ウェアハウス）---
# Snowpark DataFrame のカラム名をモデル登録時と一致させる
print("\n--- Registry 経由の推論（上位5件）---")
iso_model_ref = reg.get_model("NQ_ISOLATION_FOREST").version("v1")
sample_sp = session.create_dataframe(recent_df[FEATURE_COLS].head(5).reset_index(drop=True))
# create_dataframe は Pandas のカラム名をそのまま使うが、大文字に変換されるため
# モデル登録時の sample_input_data と一致していれば動作する
registry_result = iso_model_ref.run(sample_sp, function_name="predict")
registry_result.show()

# --- 結果表示 ---
recent_df[["MEASURE_DATE", "CALL_DROP_RATE", "PRECIPITATION_MM",
           "WIND_SPEED_KMH", "ANOMALY_SCORE", "IS_ANOMALY"]]

### Step 8 まとめ

OSS ライブラリと Snowflake の **Container Runtime** + **Model Registry** を使って、カスタム ML パイプラインを構築しました。

| モデル | ライブラリ | 用途 | カスタム要素 |
|--------|-----------|------|------------|
| **IsolationForest** | scikit-learn | 異常検知（教師なし） | `contamination` で異常割合を制御、多次元特徴量で検知 |
| **XGBRegressor** | XGBoost | トラフィック予測 | ラグ特徴量・移動平均・カレンダー・気象の特徴量エンジニアリング |

#### ML 開発サイクルの全体像

```
データ取得（Snowpark session.sql → Pandas）
     │
     ▼
特徴量エンジニアリング（Pandas）
     │
     ▼
モデル訓練（scikit-learn / XGBoost）
     │
     ▼
Model Registry に登録（reg.log_model + target_platforms=["WAREHOUSE"]）
     │
     ▼
推論（mv.run で Snowpark DataFrame を渡す → ウェアハウスで実行）
     │
     ▼
Task + Alert で自動化（発展）
```

> **ポイント**: `target_platforms=["WAREHOUSE"]` を指定して登録すると、ウェアハウス上で `mv.run()` による推論が実行できます。  
> **発展**: Model Registry にはバージョン管理機能があるため、モデルを改善したら `v2`, `v3` と登録し、  
> `DEFAULT_VERSION` を切り替えることで **ダウンタイムなし** でモデルを更新できます。